In [ ]:
# Monta o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define o caminho para o arquivo zip no seu Drive
# Substitua 'caminho/para/seu/arquivo.zip' pelo caminho real do seu arquivo
zip_file_path = '/content/drive/MyDrive/dataset/dataset.zip'

# Define o diretório de destino para extrair os arquivos
extract_path = '/content/'

# Cria o diretório de destino se ele não existir
import os
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Descompacta o arquivo zip
# Usa um comando shell para descompactar. O '!' permite executar comandos shell no Colab.
print(f"Descompactando {zip_file_path} para {extract_path}...")
!unzip -q "{zip_file_path}" -d "{extract_path}"
print("Descompactação concluída.")

# Agora você pode acessar os arquivos descompactados no diretório especificado (por exemplo, '/content/dataset')
# Exemplo: listar o conteúdo do diretório extraído
print(f"\nConteúdo do diretório descompactado ({extract_path}):")
!ls "{extract_path}"

# Você pode agora usar a variável 'extract_path' nas células subsequentes
# para referenciar a localização dos seus dados. Por exemplo, para definir DATASET_DIR:
# DATASET_DIR = extract_path

In [ ]:
import os
import re
import numpy as np
import math

from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.manifold import TSNE
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
DATASET_DIR = "/content/dataset"

CLASSES = ['Encyonema', 'Eunotia', 'Gomphonema', 'Navicula', 'Pinnularia']

# Mapeamento de nome de classe para um número (label)
class_to_int = {class_name: i for i, class_name in enumerate(CLASSES)}
int_to_class = {i: class_name for i, class_name in enumerate(CLASSES)}

In [ ]:
def get_file_lists_and_groups(dataset_dir, classes):
    """
    Mapeia todos os arquivos, extrai seus labels e seus "grupos de família".
    """
    filepaths = []  # Lista de todos os caminhos de arquivo
    labels = []     # Lista de labels (0, 1, 2, 3, 4)
    groups = []     # Lista de "IDs de família" (o base_filename)

    # Regex para extrair o "base_filename"
    # Ele captura (grupo 1) o nome base e (grupo 2) o sufixo de augmentação
    # Ex: "Gomphonema_..._timestamp_horizontal.png"
    # Grupo 1: "Gomphonema_..._timestamp"
    # Grupo 2: "_horizontal"
    # Ex: "Gomphonema_..._timestamp.png"
    # Grupo 1: "Gomphonema_..._timestamp"
    # Grupo 2: None (ou string vazia)
    pattern = re.compile(
        r'(.+?)(_horizontal|_vertical|_90_graus|_270_graus)?\.(png|jpg|jpeg|bmp|tiff)$',
        re.IGNORECASE # Ignora maiúsculas/minúsculas
    )

    print(f"Mapeando arquivos em '{dataset_dir}'...")

    for class_name in classes:
        class_dir = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_dir):
            print(f"[AVISO] Pasta não encontrada: {class_dir}")
            continue

        label = class_to_int[class_name]

        for filename in os.listdir(class_dir):
            match = pattern.match(filename)

            if match:
                # O "base_filename" é nosso ID de grupo (família)
                base_filename = match.group(1)
                filepath = os.path.join(class_dir, filename)

                filepaths.append(filepath)
                labels.append(label)
                groups.append(base_filename)

    print(f"Mapeamento concluído. {len(filepaths)} arquivos encontrados.")
    return np.array(filepaths), np.array(labels), np.array(groups)


In [ ]:
# --- 2. Execução do Mapeamento ---

all_filepaths, all_labels, all_groups = get_file_lists_and_groups(DATASET_DIR, CLASSES)

# --- 3. Divisão por Grupo (Família) ---

print("\nIniciando divisão Treino/Validação por 'Família'...")

# Queremos 1 divisão (n_splits=1) com 20% dos *grupos* para teste/validação
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# gss.split() nos dá os *índices* dos arrays para treino e validação
# Usamos 'all_filepaths' como X (só para ter o tamanho), e 'all_groups' como 'groups'
train_idx, val_idx = next(gss.split(all_filepaths, groups=all_groups))

# --- 4. Criação das Listas Finais ---

# Selecionamos os arquivos e labels com base nos índices
train_files = all_filepaths[train_idx]
train_labels = all_labels[train_idx]

val_files = all_filepaths[val_idx]
val_labels = all_labels[val_idx]

print("Divisão concluída.")
print(f"  Imagens de Treino:    {len(train_files)}")
print(f"  Imagens de Validação: {len(val_files)}")

In [ ]:
# --- 5. Verificação de Vazamento (Data Leakage) ---

# Para confirmar, vamos verificar se algum 'base_filename'
# existe em *ambos* os conjuntos. A interseção deve ser 0.

train_groups_set = set(all_groups[train_idx])
val_groups_set = set(all_groups[val_idx])

leakage = train_groups_set.intersection(val_groups_set)

if not leakage:
    print("\n[SUCESSO] Verificação de vazamento concluída. Nenhuma família de imagens vazou entre os conjuntos de Treino e Validação.")
else:
    print(f"\n[ERRO] Verificação de vazamento falhou! {len(leakage)} famílias estão em ambos os conjuntos.")

In [ ]:
# --- 6. Cálculo dos Pesos de Classe (Class Weights) ---

print("\nCalculando pesos de classe para lidar com desbalanceamento...")

# É importante calcular os pesos com base APENAS nos dados de TREINO,
# pois é neles que o modelo será treinado.
y_integers = train_labels

# Obter as classes únicas presentes nos dados de treino
classes_unicas = np.unique(y_integers)
print(f"Classes únicas encontradas nos dados de treino: {classes_unicas}")

# Calcular os pesos
# O modo 'balanced' faz o cálculo automaticamente: N_amostras / (N_classes * N_amostras_por_classe)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=classes_unicas,
    y=y_integers
)

# Criar o dicionário de pesos que o Keras espera (ex: {0: 1.5, 1: 0.8, ...})
class_weights = dict(zip(classes_unicas, class_weights_array))

print("Cálculo de pesos concluído.")
print("Estes pesos serão usados para penalizar erros em classes minoritárias:")

# Imprimir os pesos de forma legível
# Lembre-se: int_to_class = {0: 'Encyonema', 1: 'Eunotia', ...}
# (Se você não tiver o 'int_to_class', pode imprimir o dicionário 'class_weights' diretamente)
for class_int, weight in class_weights.items():
    class_name = int_to_class.get(class_int, "Classe Desconhecida")
    print(f"  -> Classe {class_int} ({class_name}): Peso = {weight:.4f}")

In [ ]:
# --- 7. Definição do Pipeline de Dados (com Augmentação e Rotação) ---

# Parâmetros
IMAGE_SIZE = 400
NUM_CHANNELS = 3
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE # Otimização do TensorFlow

def load_image(filepath, label):
    """
    Carrega, decodifica, converte (Grayscale->RGB) e redimensiona.
    Saída: pixels no intervalo [0, 255]
    """
    image = tf.io.read_file(filepath)
    image = tf.io.decode_png(image, channels=1)
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    return image, label

def augment_image(image, label):
    """
    Aplica augmentações aleatórias em tempo real.
    Entrada/Saída: pixels no intervalo [0, 255]
    """
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    k_rot = tf.random.uniform(shape=(), minval=0, maxval=4, dtype=tf.int32)
    image = tf.image.rot90(image, k=k_rot)
    image = tf.image.random_brightness(image, max_delta=0.1 * 255.0)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    image = tf.clip_by_value(image, 0.0, 255.0)
    return image, label

def normalize_image(image, label):
    """
    Normaliza a imagem para o formato que a ResNetV2 espera [-1, 1].
    """
    image = tf.keras.applications.resnet_v2.preprocess_input(image)
    return image, label

def create_dataset(filepaths, labels, is_training=True):
    """
    Cria um objeto tf.data.Dataset completo.
    """
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    if is_training:
        # 1. Embaralhar os FILEPATHS (strings), o que é leve para a RAM.
        dataset = dataset.shuffle(buffer_size=len(filepaths), reshuffle_each_iteration=True)

    # 2. Carregar as imagens (agora em ordem aleatória)
    dataset = dataset.map(load_image, num_parallel_calls=AUTOTUNE)

    if is_training:
        # 3. Aplicar a augmentação
        dataset = dataset.map(augment_image, num_parallel_calls=AUTOTUNE)

    # 4. Agrupar em lotes (Batch)
    dataset = dataset.batch(BATCH_SIZE)

    # 5. Normalizar (depois do batch, é mais rápido na GPU)
    dataset = dataset.map(normalize_image, num_parallel_calls=AUTOTUNE)

    # 6. Otimização: Pré-carregar o próximo lote
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)

    return dataset

print("\nCriando pipelines de dados com AUGMENTAÇÃO ONLINE...")

In [ ]:
# --- 8. Criar os Datasets Finais ---

# Converter labels para o formato correto (one-hot encoding)
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels, num_classes=len(CLASSES))
val_labels_one_hot = tf.keras.utils.to_categorical(val_labels, num_classes=len(CLASSES))

# Criar os pipelines
train_dataset = create_dataset(train_files, train_labels_one_hot, is_training=True)
val_dataset = create_dataset(val_files, val_labels_one_hot, is_training=False) # Validação NUNCA é aumentada

print("Pipelines criados com sucesso.")
print(f"  -> train_dataset (com augmentação): {train_dataset}")
print(f"  -> val_dataset (sem augmentação):   {val_dataset}")

In [ ]:
# --- 9. Definição da Arquitetura do Modelo ---

print("\nConstruindo o modelo de Transfer Learning (ResNet50V2)...")

INPUT_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS) # (400, 400, 3)
NUM_CLASSES = len(CLASSES) # 5

# --- Parte 1: Carregar a Base Pré-Treinada ---

# Carregamos a ResNet50V2, treinada no ImageNet
# include_top=False: remove a camada final original (que classificava 1000 classes)
base_model = ResNet50V2(
    weights='imagenet',
    include_top=False,
    input_shape=INPUT_SHAPE
)

# --- Parte 2: Congelar a Base ---

# "Congelar" os pesos da base.
# Não queremos re-treiná-los na primeira fase (Feature Extraction).
base_model.trainable = False

# --- Parte 3: Construir o "Head" (Nosso Classificador) ---

# 1. Definir a entrada do nosso modelo
inputs = Input(shape=INPUT_SHAPE)

# 2. Passar a entrada pela base (em modo "inferência")
# training=False garante que as camadas de BatchNormalization da base
# usem suas estatísticas salvas e não tentem se atualizar.
x = base_model(inputs, training=False)

# 3. Adicionar nossas camadas no topo
# GlobalAveragePooling2D achata a saída 4D da ResNet em um vetor 1D
x = GlobalAveragePooling2D(name='global_average_pooling2d')(x)
# Adicionamos uma camada densa para aprender os padrões específicos das diatomáceas
x = Dense(256, activation='relu', name='dense_head')(x)
# Dropout é uma técnica de regularização crucial para evitar overfitting
x = Dropout(0.5)(x)
# Camada de saída final. 'softmax' para classificação multiclasse
outputs = Dense(NUM_CLASSES, activation='softmax', name='predictions')(x)

# --- Parte 4: Criar o Modelo Final ---
model = Model(inputs, outputs)

# --- Parte 5: Compilar o Modelo ---
model.compile(
    # Adam é um otimizador robusto. 0.001 é um bom learning rate inicial
    optimizer=Adam(learning_rate=0.001),
    # 'categorical_crossentropy' é a loss function correta
    # porque usamos to_categorical (one-hot) em nossos labels.
    loss='categorical_crossentropy',
    metrics=['accuracy'] # Vamos monitorar a acurácia
)

print("Modelo construído e compilado com sucesso.")

# --- 10. Exibir Resumo do Modelo ---
print("\nResumo do modelo:")
model.summary()

In [ ]:
# --- 11. Configuração dos Callbacks ---

print("\nConfigurando callbacks (ModelCheckpoint e EarlyStopping)...")

# Define o caminho para salvar o melhor modelo
best_model_path = "/content/drive/MyDrive/dataset/diatom_classifier_best_model_puroTratado.keras"

# 1. ModelCheckpoint: Salva o modelo com a melhor acurácia de validação
model_checkpoint = ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,       # Salva apenas se for melhor que o anterior
    monitor='val_accuracy',    # Monitora a acurácia da validação
    mode='max',                # Queremos maximizar a acurácia
    verbose=1                  # Imprime uma mensagem quando salva
)

# 2. EarlyStopping: Para o treinamento se não houver melhoria
early_stopping = EarlyStopping(
    monitor='val_loss',        # Monitora a perda da validação
    patience=3,                # Número de épocas sem melhoria antes de parar (3 é um bom começo)
    mode='min',                # Queremos minimizar a perda
    verbose=1,
    restore_best_weights=True  # Restaura os pesos da melhor época ao final
)

In [ ]:
# --- Função de Plotagem das Curvas de Treinamento ---

def plot_training_curves(history, title_suffix):
    """
    Plota as curvas de Acurácia e Perda do treinamento.

    Argumentos:
        history (tf.keras.callbacks.History): O objeto retornado por model.fit().
        title_suffix (str): O sufixo para o título (ex: "Fase 1" ou "Fase 2").
    """
    print(f"\nGerando curvas de treinamento para: {title_suffix}...")

    # Pega as métricas do histórico
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Gráfico de Acurácia
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Acurácia (Treino)')
    plt.plot(epochs, val_acc, 'ro-', label='Acurácia (Validação)')
    plt.title(f'Curva de Acurácia - {title_suffix}')
    plt.xlabel('Épocas')
    plt.ylabel('Acurácia')
    plt.legend()
    plt.grid(True)

    # Gráfico de Perda (Loss)
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Perda (Treino)')
    plt.plot(epochs, val_loss, 'ro-', label='Perda (Validação)')
    plt.title(f'Curva de Perda - {title_suffix}')
    plt.xlabel('Épocas')
    plt.ylabel('Perda')
    plt.legend()
    plt.grid(True)

    plt.suptitle(f'Curvas de Treinamento - {MODEL_NAME}', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

In [ ]:

# --- 12. Início do Treinamento (Fase 1) ---

print("\n--- INICIANDO TREINAMENTO (FASE 1: FEATURE EXTRACTION) ---")

NUM_EPOCHS = 20  # Vamos começar com 20 épocas para esta fase
BATCH_SIZE = 32  # O BATCH_SIZE que definimos no pipeline

# Calcular os "steps" (passos) por época.
# Isso é necessário ao usar tf.data
steps_per_epoch = math.ceil(len(train_files) / BATCH_SIZE)
validation_steps = math.ceil(len(val_files) / BATCH_SIZE)

history = model.fit(
    train_dataset,
    epochs=NUM_EPOCHS,
    validation_data=val_dataset,
    class_weight=class_weights,  # <-- Aplicando os pesos contra o desbalanceamento!
    callbacks=[model_checkpoint, early_stopping],
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

print("\n--- TREINAMENTO (FASE 1) CONCLUÍDO ---")
print(f"O melhor modelo foi salvo em: {best_model_path}")

try:
    plot_training_curves(history, f"Fase 1 - {MODEL_NAME}")
except NameError:
    plot_training_curves(history, "Fase 1: Extração de Features")

In [ ]:
# --- 13. Carregar o Modelo da Fase 1 ---

print("\n--- INICIANDO FASE 2: AJUSTE FINO (FINE-TUNING) ---")
print("Carregando o melhor modelo da Fase 1...")

# Carrega o modelo que atingiu melhor acurácia
# Keras também carrega o estado do otimizador, mas vamos recompilar.
model = load_model('/content/drive/MyDrive/dataset/diatom_classifier_best_model_puroTratado.keras')

# --- 14. "Descongelar" a Base ---

# Precisamos acessar a "base_model" (a ResNet) dentro do nosso modelo salvo
# O nome 'resnet50v2' é o nome padrão que Keras deu a ela (vimos no model.summary())
try:
    base_model = model.get_layer('resnet50v2')
    base_model.trainable = True # <-- A MÁGICA ACONTECE AQUI
    print(f"Camada '{base_model.name}' foi descongelada e está pronta para o ajuste fino.")
except ValueError:
    print("ERRO: Não foi possível encontrar a camada 'resnet50v2'. Verifique o model.summary() da Fase 1.")
    # Se o nome for diferente, ajuste-o.

# --- 15. Recompilar com Taxa de Aprendizado Baixíssima ---

# Este é o passo MAIS CRÍTICO do ajuste fino.
# Usamos uma taxa de aprendizado 100x a 1000x menor que antes.
# Queremos "ajustar" os pesos, não "destruí-los".
LEARNING_RATE_PHASE_2 = 0.00001 # (1e-5)

model.compile(
    # Adam com um learning rate muito baixo
    optimizer=Adam(learning_rate=LEARNING_RATE_PHASE_2),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Modelo recompilado para ajuste fino com learning rate de {LEARNING_RATE_PHASE_2}.")

# Vamos verificar o resumo. Agora, TODOS os parâmetros devem ser "Trainable".
print("\nResumo do modelo para Fase 2:")
model.summary()

# --- 16. Configurar Callbacks para a Fase 2 ---

# Define o caminho para salvar o melhor modelo no seu Drive
best_model_path_phase2 = "/content/drive/MyDrive/dataset/diatom_classifier_best_model_puroTratado_finetuned.keras"

# 1. ModelCheckpoint para a Fase 2
model_checkpoint_phase2 = ModelCheckpoint(
    filepath=best_model_path_phase2,
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

# 2. EarlyStopping para a Fase 2 (com paciência, pois as melhorias são lentas)
early_stopping_phase2 = EarlyStopping(
    monitor='val_loss',
    patience=3,  # 3 épocas sem melhoria na perda de validação
    mode='min',
    verbose=1,
    restore_best_weights=True
)

In [ ]:
# --- 17. Início do Treinamento (Fase 2) Fine-Tuning ---

print("\n--- CONTINUANDO TREINAMENTO (FASE 2: AJUSTE FINO) ---")

# Vamos treinar por mais épocas. O modelo continuará de onde parou.
NUM_EPOCHS_PHASE_2 = 20 # Vamos dar mais 10 épocas de chance
BATCH_SIZE = 32 # O mesmo BATCH_SIZE

# Reutilizar os steps calculados na Fase 1
# (Assumindo que 'train_files' e 'val_files' ainda estão na memória)
try:
    steps_per_epoch = math.ceil(len(train_files) / BATCH_SIZE)
    validation_steps = math.ceil(len(val_files) / BATCH_SIZE)
except NameError:
    print("Recalculando steps (variáveis não encontradas)...")
    # Se você estiver em um novo script, precisará recriar as listas
    # de arquivos para obter o len() delas.
    # Por segurança (assumindo que você está no mesmo notebook):
    steps_per_epoch = 551
    validation_steps = 138

history_phase2 = model.fit(
    train_dataset,
    epochs=NUM_EPOCHS_PHASE_2,
    validation_data=val_dataset,
    class_weight=class_weights,  # <-- Ainda usamos os pesos de classe!
    callbacks=[model_checkpoint_phase2, early_stopping_phase2],
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
    # O modelo magicamente "continua" porque carregamos seus pesos.
)

print("\n--- TREINAMENTO (FASE 2) CONCLUÍDO ---")
print(f"O melhor modelo de ajuste fino foi salvo em: {best_model_path_phase2}")

try:
    plot_training_curves(history_phase2, f"Fase 2 - {MODEL_NAME}")
except NameError:
    plot_training_curves(history_phase2, "Fase 2: Ajuste Fino")

In [ ]:
def plot_tsne_visualization(model, val_dataset, y_true_labels, classes, model_name):
    """
    Extrai features (embeddings) do modelo, aplica t-SNE e plota a visualização 2D.

    Argumentos:
        model (tf.keras.Model): O modelo treinado e carregado.
        val_dataset (tf.data.Dataset): O pipeline de dados de validação.
        y_true_labels (np.array): O array de labels verdadeiros (como índices, ex: [0, 1, 4...]).
        classes (list): A lista de nomes das classes (ex: ['Encyonema', ...]).
        model_name (str): O nome do modelo para o título do gráfico.
    """
    print("\n" + "="*50)
    print("--- PASSO 10: Gerando Visualização t-SNE ---")
    print("="*50)

    # 1. Criar o "extrator de features"
    # Usamos o nome 'global_average_pooling2d' que acabamos de adicionar
    try:
        feature_layer = model.get_layer('global_average_pooling2d')
    except ValueError:
        print("ERRO: Não foi possível encontrar a camada 'global_average_pooling2d'.")
        print("Certifique-se de que você nomeou a camada e re-treinou o modelo.")
        return

    # Cria um novo modelo que termina na camada de features
    feature_extractor = Model(inputs=model.input, outputs=feature_layer.output)

    # 2. Extrair as features (embeddings) de todo o dataset de validação
    print("Extraindo features (embeddings) do dataset de validação...")
    # A função .predict() irá iterar por todo o val_dataset
    features = feature_extractor.predict(val_dataset, verbose=1)
    # O resultado será (1468, 2048) - (N_imagens_val, N_features)
    print(f"Features extraídas. Shape: {features.shape}")

    # 3. Rodar o t-SNE
    # Isso pode levar alguns minutos.
    print("Calculando t-SNE (isso pode levar alguns minutos)...")
    tsne = TSNE(n_components=2,       # Reduzir para 2 dimensões (x, y)
                perplexity=30.0,    # Valor padrão
                n_iter=1000,        # Iterações
                random_state=42,    # Reprodutibilidade
                verbose=1)          # Mostrar progresso

    tsne_results = tsne.fit_transform(features)
    # O resultado será (1468, 2)
    print("Cálculo do t-SNE concluído.")

    # 4. Plotar os resultados
    plt.figure(figsize=(14, 10))
    for i, class_name in enumerate(classes):
        # Encontra os índices (posições) de todas as imagens que pertencem a esta classe
        indices = np.where(y_true_labels == i)

        # Plota um scatter plot apenas para esses pontos
        plt.scatter(tsne_results[indices, 0], tsne_results[indices, 1],
                    label=class_name,
                    alpha=0.7,
                    s=15)

    plt.title(f'Visualização t-SNE das Features Extraídas - {MODEL_NAME}')
    plt.xlabel('Componente t-SNE 1')
    plt.ylabel('Componente t-SNE 2')
    plt.legend(markerscale=3) # Legendas com marcadores maiores
    plt.show()

In [ ]:
# --- 18. Carregar o Modelo Final ---
print("\n--- INICIANDO FASE FINAL: AVALIAÇÃO ---")

# Use o nome do arquivo da Fase 2 (fine-tuned)
MODEL_PATH = '/content/drive/MyDrive/dataset/diatom_classifier_best_model_finetuned.keras'
MODEL_NAME = 'Modelo Tratado 1.0 (99.02%)' # Dê um nome para os gráficos

print(f"Carregando o modelo final de: {MODEL_PATH}")
model = load_model(MODEL_PATH)

In [ ]:
# --- 19. Avaliação Final Simples ---
print("\nAvaliando o modelo final no dataset de validação...")
# (val_dataset, val_files, BATCH_SIZE vêm das células anteriores)
validation_steps = math.ceil(len(val_files) / BATCH_SIZE)
results = model.evaluate(val_dataset, steps=validation_steps, verbose=1)

print("\nResultados da Avaliação Final:")
print(f"  Perda (Loss):    {results[0]:.4f}")
print(f"  Acurácia (Acc): {results[1]*100:.2f}%")

# --- 20. Obter Predições vs. Labels Reais (para as próximas células) ---
print("\nGerando predições em todo o dataset de validação...")

# Recria os labels one-hot (caso o notebook tenha sido reiniciado)
val_labels_one_hot = tf.keras.utils.to_categorical(val_labels, num_classes=len(CLASSES))
# Recria o dataset de validação (sem shuffle)
val_dataset_eval = create_dataset(val_files, val_labels_one_hot, is_training=False)

# Listas para armazenar os resultados
y_pred_proba_list = []
y_true_labels_list = []

# Itera pelo dataset para pegar predições e labels
for images, labels in val_dataset_eval:
    y_pred_proba_list.append(model.predict(images, verbose=0))
    y_true_labels_list.extend(np.argmax(labels.numpy(), axis=1))

# Converte as listas em arrays NumPy
y_pred_proba = np.concatenate(y_pred_proba_list, axis=0) # Probabilidades (ex: [0.1, 0.9, ...])
y_true_labels = np.array(y_true_labels_list)           # Labels de índice (ex: [1, 1, ...])
y_true_one_hot = val_labels_one_hot                      # Labels One-Hot (para ROC)
y_pred_labels = np.argmax(y_pred_proba, axis=1)        # Labels de índice (ex: [1, 2, ...])

print("Predições concluídas e armazenadas em variáveis.")

# --- 21. Relatório de Classificação ---
print("\n--- Relatório de Classificação (Precision, Recall, F1-Score) ---")
print(classification_report(y_true_labels, y_pred_labels, target_names=CLASSES))

# --- 22. Matriz de Confusão ---
print("\nGerando Matriz de Confusão...")
cm = confusion_matrix(y_true_labels, y_pred_labels)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=CLASSES, yticklabels=CLASSES)
plt.title(f'Matriz de Confusão - Acurácia: {results[1]*100:.2f}% - {MODEL_NAME}')
plt.ylabel('Classe Verdadeira (True Label)')
plt.xlabel('Classe Prevista (Predicted Label)')
plt.show()

In [ ]:
# --- Curvas ROC e AUC ---
print("\n" + "="*50)
print("--- PASSO 9: Gerando Curvas ROC e AUC (One-vs-Rest) ---")
print("="*50)

# Dicionários para armazenar as taxas
fpr = dict()
tpr = dict()
roc_auc = dict()
n_classes = len(CLASSES)

plt.figure(figsize=(10, 8))

# Binariza os labels verdadeiros (necessário para One-vs-Rest)
# y_true_one_hot já foi carregado na célula anterior

# Calcula a curva ROC e AUC para cada classe
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_one_hot[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    plt.plot(fpr[i], tpr[i], lw=2,
             label=f'Classe {CLASSES[i]} (AUC = {roc_auc[i]:.4f})')

# Plota a linha de "chute aleatório"
plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Chance (AUC = 0.50)')

# Formatação do gráfico
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos (1 - Especificidade)')
plt.ylabel('Taxa de Verdadeiros Positivos (Recall)')
plt.title(f'Curvas ROC (One-vs-Rest) - {MODEL_NAME}')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

In [ ]:
# --- Visualização t-SNE ---
print("\n" + "="*50)
print("--- PASSO 10: Gerando Visualização t-SNE ---")
print("="*50)

# 1. Criar o "extrator de features"
# Usamos o nome 'global_average_pooling2d' que demos ao nosso modelo
try:
    feature_layer = model.get_layer('global_average_pooling2d')
except ValueError:
    print("ERRO: Não foi possível encontrar a camada 'global_average_pooling2d'.")
    print("Certifique-se de que você nomeou a camada na Célula 9 e RE-TREINOU o modelo.")

if 'feature_layer' in locals():
    # Cria um novo modelo que termina na camada de features
    feature_extractor = Model(inputs=model.input, outputs=feature_layer.output)

    # 2. Extrair as features (embeddings) de todo o dataset de validação
    print("Extraindo features (embeddings) do dataset de validação...")
    # val_dataset_eval foi criado na célula de avaliação
    features = feature_extractor.predict(val_dataset_eval, verbose=1)
    print(f"Features extraídas. Shape: {features.shape}")

    # 3. Rodar o t-SNE
    print("Calculando t-SNE (isso pode levar alguns minutos)...")
    tsne = TSNE(n_components=2,       # Reduzir para 2D
                perplexity=30.0,    # Padrão
                n_iter=1000,
                random_state=42,
                verbose=1)

    tsne_results = tsne.fit_transform(features)
    print("Cálculo do t-SNE concluído.")

    # 4. Plotar os resultados
    plt.figure(figsize=(14, 10))
    for i, class_name in enumerate(CLASSES):
        # Encontra os índices (posições) de todas as imagens que pertencem a esta classe
        # y_true_labels foi calculado na célula de avaliação
        indices = np.where(y_true_labels == i)

        plt.scatter(tsne_results[indices, 0], tsne_results[indices, 1],
                    label=class_name,
                    alpha=0.7,
                    s=15)

    plt.title(f'Visualização t-SNE das Features Extraídas - {MODEL_NAME}')
    plt.xlabel('Componente t-SNE 1')
    plt.ylabel('Componente t-SNE 2')
    plt.legend(markerscale=3)
    plt.show()